# Concatenate boreholes

In [1]:
%matplotlib widget

In [2]:
from utils.io import gen_id_dated, gdf_viewer, gdf_geom, gdf_merger
import re, os
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import datetime
import matplotlib.pyplot as plt

# Formatage des données (fichiers excel)

Regrouper les infos sur les infos par type (forages, echantillons, ...) dans des fichiers csv

## 1- Profils sols et données forages.xls
* **Feuille : 'Données de forage'**

In [3]:
tmp_dir='../../CF_data/synthese/Result_traitem/profils_sols_donnees_forages/'
sheet='donnees_forage'

In [4]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Données de forage')#, skiprows=2)
gdf_viewer(tmp_for)

Rows : 25, columns : 15


interactive(children=(IntSlider(value=10, description='rows', max=25, min=10, readout=False), IntSlider(value=…

In [5]:
tmp_for.rename(columns={'Date':'Date_ouv','Profondeur':'Long_For', 'Méthode':'Method', 
                        'Diamètre forage':'Diam_For','Niv. Eau p/r sol':'Niv_Eau_sol',
                        'PZ Prof.':'Long_Pz', 'PZ Diamètre':'Diam_Pz','PZ L.crépinée':'Long_Crep', 
                        'Société forage':'Societe'}, inplace=True)

tmp_for=tmp_for[['ID', 'X', 'Y', 'Z', 'Date_ouv', 'Long_For', 'Diam_For', 'Long_Pz', 'Diam_Pz', 'Long_Crep',
                 'Remarque','Niv_Eau_sol','Method', 'Societe']]

In [6]:
tmp_for['Type'] = tmp_for['Long_Pz'].apply(lambda x: '' if pd.isnull(x) else 'Piezo')
tmp_for['Refus'] = ''
for i in range(len(tmp_for['Remarque'])):
    val = str(tmp_for.loc[i,'Remarque'])
    if re.search('[Bb]loqué', val) and re.search('[lL]aitier', val):
        tmp_for.loc[i,'Refus'] = 'Laitier'
    elif re.search('[Bb]loqué', val) and re.search('[Bb]éton', val):
        tmp_for.loc[i,'Refus'] = 'Béton'
    elif re.search('[Bb]loqué', val) and re.search('[Mm]atériaux', val):
        tmp_for.loc[i,'Refus'] = 'Matériaux indurés' 
    else: 
        tmp_for.loc[i,'Refus'] = '' 

tmp_for['Diam_Pz_int'] = tmp_for['Diam_Pz'].apply(lambda x: pd.to_numeric(x.replace(' mm','').split('x')[1].strip(' m'))/1000 
                                                  if not pd.isnull(x) else x)
tmp_for['Diam_Pz_ext'] = tmp_for['Diam_Pz'].apply(lambda x: pd.to_numeric(x.replace(' mm','').split('x')[0].strip(' m'))/1000 
                                                  if not pd.isnull(x) else x)
tmp_for.insert(7, 'Diam_Pz_ext', tmp_for.pop('Diam_Pz_ext')) # move to a specified position
tmp_for.insert(8, 'Diam_Pz_int', tmp_for.pop('Diam_Pz_int'))
tmp_for.drop(columns=['Remarque', 'Diam_Pz'], axis=1, inplace=True)
tmp_for.drop(tmp_for.query("ID!=ID").index, inplace=True) # delete all ID='NaN' lines

gen_id_dated(tmp_for,'ID','Date_ouv')  

Generation of ID-dated...
Using column ' Date_ouv ' in the (geo)dataframe !
Process ended, check you (geo)dataframe


In [7]:
gdf_viewer(tmp_for, cols=12)

Rows : 25, columns : 17


interactive(children=(IntSlider(value=10, description='rows', max=25, min=10, readout=False), IntSlider(value=…

In [8]:
source_for=tmp_for.copy()

In [9]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
tmp_for.to_csv(tmp_dir+sheet+'.csv', index=False)
source_for.to_csv(tmp_dir+'source_Boreholes.csv', index=False) #all Boreholes/piezo and trenches data in the source

* **Feuille : 'Piézométrie'**

In [10]:
tmp_dir='../../CF_data/synthese/Result_traitem/profils_sols_donnees_forages/'
sheet='piezometrie'

In [11]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Piézométrie', skiprows=1)
gdf_viewer(tmp_for)

Rows : 37, columns : 21


interactive(children=(IntSlider(value=10, description='rows', max=37, min=10, readout=False), IntSlider(value=…

In [12]:
merge_tmp=tmp_for[:11].copy()
merge_tmp.reset_index(inplace=True, drop=True)
merge_tmp.rename(columns={'z':'Z',}, inplace=True)
merge_tmp=merge_tmp[['ID', 'Z']]
#merge_tmp['Type']='Piezo'

gdf_viewer(merge_tmp, un_val='ID')

Rows : 11, columns : 2, Unique on 'ID': 11


interactive(children=(IntSlider(value=10, description='rows', max=11, min=10, readout=False), IntSlider(value=…

In [13]:
a=0
for x in tmp_for.columns:
    if pd.isnull(tmp_for.loc[16,x]):
        tmp_for.loc[16,x]='col'+str(a)
    a+=1

In [14]:
tmp_for.loc[16]=tmp_for.loc[16].apply(lambda x : x if not pd.isnull(x) else '')
tmp_for.columns = tmp_for.loc[16]
tmp_for=tmp_for[17:]
#tmp_for.rename_axis(None, inplace=True)
tmp_for.reset_index(inplace=True, drop=True)

tmp_for.drop(columns=[tmp_for.columns.to_list()[x] for x in range(0,8)
                      if re.compile(r"col|unnamed").match(tmp_for.columns.to_list()[x])], axis=1, inplace=True) 

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
tmp_for.rename(columns={'col8':'Date_prv', 'col9':'Terrain', 'col10':'ID', 'NP/piézo [m]':'Niv_Eau_Pz', 
                        'dim. piezo hors sol [m]':'haut_Pz_sol', 'NP/sol [m]':'Niv_Eau_sol', 
                        'Prof. piézo/piézo [m]':'Long_Pz', 'Prof. piézo/sol [m]':'Long_Pz_sol', 
                        'CE [mS/cm]':'CE','t° [°C]':'Temp', 'Observations':'Obsv'}, inplace=True)

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [16]:
gdf_viewer(tmp_for)

Rows : 20, columns : 13


interactive(children=(IntSlider(value=10, description='rows', max=20, min=10, readout=False), IntSlider(value=…

In [17]:
tmp_for['Type'] = tmp_for['Long_Pz'].apply(lambda x: '' if pd.isnull(x) else 'Piezo')
tmp_for.insert(0, 'ID', tmp_for.pop('ID')) # move to first column
tmp_for.replace('-', np.nan, inplace=True)
tmp_for['CE']=tmp_for['CE [µS/cm]'].apply(lambda x: x/1000 
                                    if isinstance(x, float) and not pd.isnull(x) else np.nan) #
tmp_for.drop('CE [µS/cm]', axis=1, inplace=True)
tmp_for.loc[18, 'Niv_Eau_Pz']=np.nan
#tmp_for['Date_prv']=tmp_for['Date_prv'].apply(lambda x : str(x.year) if not pd.isnull(x) else '')

<ipython-input-17-767abb4af3b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Type'] = tmp_for['Long_Pz'].apply(lambda x: '' if pd.isnull(x) else 'Piezo')
/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/frame.py:4521: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-17-767abb4af3b4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [18]:
merge_tmp.merge(tmp_for, how='inner', left_on='ID', right_on='ID')

,ID,Z,Date_prv,Terrain,Niv_Eau_Pz,haut_Pz_sol,Niv_Eau_sol,Long_Pz,Long_Pz_sol,pH,CE,Temp,Obsv,Type


In [19]:
merge_tmp=tmp_for.append(merge_tmp)

In [20]:
merge_tmp.reset_index(inplace=True, drop=True)
merge_tmp.drop(merge_tmp.query("ID!=ID").index, inplace=True) # supprimer les lignes avec ID='NaN'
merge_tmp.reset_index(inplace=True, drop=True)
gdf_viewer(merge_tmp, un_val='ID')

Rows : 28, columns : 14, Unique on 'ID': 28


interactive(children=(IntSlider(value=10, description='rows', max=28, min=10, readout=False), IntSlider(value=…

 ------------------------ Merge with **'source_..._data'** (sheets data merge into one file for each source) ----------------------

In [21]:
source_for, error_df=gdf_merger(source_for, merge_tmp, how='outer', col='ID')

In [22]:
gdf_viewer(source_for, un_val='ID', rows=3) #source_df is a merge of all data treated in a source

Rows : 42, columns : 26, Unique on 'ID': 42


interactive(children=(IntSlider(value=3, description='rows', max=42, min=3, readout=False), IntSlider(value=12…

In [23]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
merge_tmp.to_csv(tmp_dir+sheet+'.csv', index=False)
source_for.to_csv(tmp_dir+'source_Boreholes.csv', index=False) #all Boreholes/piezo and trenches data in the source

* **Feuille : 'Equipement'**

In [24]:
tmp_dir='../../CF_data/synthese/Result_traitem/profils_sols_donnees_forages/'
sheet='Equipement'

In [25]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Equipement')#, skiprows=1)
gdf_viewer(tmp_for)

Rows : 36, columns : 7


interactive(children=(IntSlider(value=10, description='rows', max=36, min=10, readout=False), IntSlider(value=…

In [26]:
tmp_for.rename(columns={'Diam. Forage':'Diam_For', 'Diamètre ext. PZ':'Diam_ext_Pz'}, inplace=True)
tmp_for[['Crep_top','Crep_base','Bent_top','Bent_base']]=np.nan

In [27]:
for idx, row in tmp_for.iterrows():
    if row['Légende']=='Bentonite':
        tmp_for.loc[idx,'Bent_top']=row['De']
        tmp_for.loc[idx,'Bent_base']=row['A']
    elif row['Légende']=="Crépine":
        tmp_for.loc[idx,'Crep_top']=row['De']
        tmp_for.loc[idx,'Crep_base']=row['A']

In [28]:
i=0
for idx in range(len(tmp_for)-1):
    i+=1
    if tmp_for.loc[idx,'ID']==tmp_for.loc[i,'ID'] and pd.isnull(tmp_for.loc[idx,'Crep_top']) and not pd.isnull(tmp_for.loc[i,'Crep_top']):
        tmp_for.loc[idx,'Crep_top']=tmp_for.loc[i,'Crep_top']
        tmp_for.loc[idx,'Crep_base']=tmp_for.loc[i,'Crep_base']
        
    elif tmp_for.loc[idx,'ID']==tmp_for.loc[i,'ID'] and pd.isnull(tmp_for.loc[idx,'Bent_top']) and not pd.isnull(tmp_for.loc[i,'Bent_top']):
        tmp_for.loc[idx,'Bent_top']=tmp_for.loc[i,'Bent_top']
        tmp_for.loc[idx,'Bent_base']=tmp_for.loc[i,'Bent_base']

In [29]:
tmp_for=tmp_for[['ID','Diam_For','Diam_ext_Pz','Crep_top','Crep_base','Bent_top','Bent_base']]
tmp_for.drop_duplicates('ID', inplace=True)
tmp_for.reset_index(inplace=True, drop=True)
tmp_for['Type']='Piezo'

<ipython-input-29-3dfe015cc0a3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for.drop_duplicates('ID', inplace=True)
<ipython-input-29-3dfe015cc0a3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Type']='Piezo'


In [30]:
gdf_viewer(tmp_for, un_val='ID', rows=3)

Rows : 12, columns : 8, Unique on 'ID': 12


interactive(children=(IntSlider(value=3, description='rows', max=12, min=3, readout=False), IntSlider(value=8,…

 ------------------------ Merge with **'source_..._data'** (sheets data merge into one file for each source) ----------------------

In [31]:
source_for, error_df = gdf_merger(source_for, tmp_for, how='outer', col='ID')

Ambiguous values in both columns compared, change it manually !
Columns ['Diam_For_x', 'Diam_For_y'] [16, 19, 20, 24] must be dropped manually !

Creation of a dataframe for ambiguous values, check it !


In [32]:
gdf_viewer(source_for), gdf_viewer(error_df)

Rows : 42, columns : 33


interactive(children=(IntSlider(value=10, description='rows', max=42, min=10, readout=False), IntSlider(value=…

Rows : 4, columns : 3


interactive(children=(IntSlider(value=4, description='rows', max=4, min=4, readout=False), IntSlider(value=3, …

(None, None)

--------->  Une verification à faire ici (en haut)

In [33]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
merge_tmp.to_csv(tmp_dir+sheet+'.csv', index=False)
source_for.to_csv(tmp_dir+'source_Boreholes.csv', index=False) #all Boreholes/piezo and trenches data in the source

* **Feuilles : 'Echantillon' + 'Organoleptique'**

In [34]:
tmp_dir='../../CF_data/synthese/Result_traitem/profils_sols_donnees_forages/'
sheet='Echant-organo'

In [35]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Echantillon')#, skiprows=1)
gdf_viewer(tmp_for)

Rows : 29, columns : 4


interactive(children=(IntSlider(value=10, description='rows', max=29, min=10, readout=False), IntSlider(value=…

In [36]:
tmp_for.rename(columns={'De':'Ech_top', 'A':'Ech_base', 'Numéro':'Ech_ID'}, inplace=True)

In [37]:
merge_tmp = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Organoleptique')#, skiprows=1)
gdf_viewer(merge_tmp)

Rows : 7, columns : 5


interactive(children=(IntSlider(value=7, description='rows', max=7, min=7, readout=False), IntSlider(value=5, …

In [38]:
list(merge_tmp.columns)

['ID', 'De', 'A', 'Polluant', 'Intensité']

In [39]:
merge_tmp.rename(columns={'De':'Pol_top', 'A':'Pol_base'}, inplace=True)

In [40]:
merge_tmp, error_df = gdf_merger(tmp_for, merge_tmp, 'outer', 'ID')
merge_tmp['Type_ech']='Sol'
merge_tmp.insert(4, 'Type_ech', merge_tmp.pop('Type_ech'))

In [41]:
gdf_viewer(merge_tmp)

Rows : 32, columns : 9


interactive(children=(IntSlider(value=10, description='rows', max=32, min=10, readout=False), IntSlider(value=…

In [42]:
source_ech=merge_tmp.copy()

In [43]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
merge_tmp.to_csv(tmp_dir+sheet+'.csv', index=False)
source_ech.to_csv(tmp_dir+'Source_Samples.csv', index=False) #all Samples and organoleptic data in the source

In [44]:
gdf_viewer(source_ech, un_val='ID')

Rows : 32, columns : 9, Unique on 'ID': 15


interactive(children=(IntSlider(value=10, description='rows', max=32, min=10, readout=False), IntSlider(value=…

* **Feuille : 'Log'**

In [45]:
tmp_dir='../../CF_data/synthese/Result_traitem/profils_sols_donnees_forages/'
sheet='Log'

In [46]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/ouvrages/Profils sols et données forages.xls', sheet_name='Log')#, skiprows=1)
gdf_viewer(tmp_for)

Rows : 55, columns : 5


interactive(children=(IntSlider(value=10, description='rows', max=55, min=10, readout=False), IntSlider(value=…

In [47]:
tmp_for.rename(columns={'De':'Litho_top', 'A':'Litho_base'}, inplace=True)

In [48]:
q=tmp_for.query('Keyword.str.contains(".ointe", regex=True)', engine='python').index
tmp_for.drop(q, inplace=True)
tmp_for.reset_index(drop=True, inplace=True)

In [49]:
gdf_viewer(tmp_for)

Rows : 54, columns : 5


interactive(children=(IntSlider(value=10, description='rows', max=54, min=10, readout=False), IntSlider(value=…

In [50]:
source_litho=tmp_for.copy()

In [51]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
tmp_for.to_csv(tmp_dir+sheet+'.csv', index=False)
source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False) #all lithologies or descriptions data in the source

=============  attempt to create a lexicon from description ===============================

A function must be write for this operation

In [52]:
kw=[]
for i in range(len(tmp_for)):
    kw=kw+tmp_for.loc[i, 'Description'].split(' ')
kw=list(set(kw))
print(kw)

['', 'couleur', 'arable', 'déchets', 'pierreuse,', 'petite', '(briquaillons', '4', 'pollué.', "d'eau", 'foncé/noir', 'vert/ocre.', 'base.', 'argileux', "d'égout.", 'beige/orange', 'briquaillons', '3,6', '2,4', 'caillasse', 'irrisation', 'mélange', 'orange.', 'pierreuse', 'plus', 'odeur', 'liner.', 'verdâtre/brun', 'graveleuse', 'forte', '2.4', '2,8', 'présence', "d'eau.", 'du', 'verdâtre', 'scories', 'décehts', 'brique.', 'limoneuse', 'remblais', 'sidérurgiques', 'ocre', 'sableux', 'Entre', '4,8', 'beige', 'poussière', 'frais', 'la', 'sableuse', 'divers', 'laitier', 'hétérogène', 'hétérogènes', 'limoneux', 'sur', '-', 'graviers.', '3.3', 'centimétrique', 'cm', '0,5', 'en', 'décimétrique', 'béton', 'verdâtre.', 'cendrées', 'Strate', 'noire', 'devient', '0.6', 'charge', 'cendrée', 'limons', 'caillouteuse.', '?', 'terreux', 'Stabilisé', 'taille', 'Mélange', "l'eau.", 'Bloqué', 'foncé.', 'entre', 'graveleuse.', 'orange).', 'dans', 'puis', '2', 'orange', 'scories/limons', 'Traces', 'gris', 

In [53]:
lexic=[]
for i in range(len(kw)):
    if len(kw[i])>2 and not re.search('\d.*',kw[i]): #to eliminate all 'one letter words' and all numbers
        lexic=lexic+[re.sub("\.|l'|d'","",kw[i])]
lexic=list(set(lexic))
print(lexic)

['arable', 'couleur', 'déchets', 'pierreuse,', 'petite', '(briquaillons', 'foncé/noir', 'argileux', 'beige/orange', 'briquaillons', 'caillasse', 'irrisation', 'vert/ocre', 'mélange', 'pierreuse', 'plus', 'odeur', 'verdâtre/brun', 'graveleuse', 'forte', 'graviers', 'présence', 'verdâtre', 'orange)', 'scories', 'décehts', 'limoneuse', 'remblais', 'sidérurgiques', 'eau', 'ocre', 'sableux', 'Entre', 'beige', 'poussière', 'frais', 'sableuse', 'divers', 'laitier', 'hétérogène', 'hétérogènes', 'limoneux', 'sur', 'centimétrique', 'décimétrique', 'béton', 'cendrées', 'Strate', 'noire', 'caillouteuse', 'devient', 'charge', 'cendrée', 'limons', 'terreux', 'Stabilisé', 'taille', 'Mélange', 'Bloqué', 'entre', 'dans', 'puis', 'scories/limons', 'orange', 'signe', 'Traces', 'gris', 'Alluvions', 'ocre/orange', 'strate', 'induré', 'limon', 'hétérogène,', 'noir', 'graviers/scories', 'base', 'schiste,', 'hydrocarbures', 'pierreux', 'Irrisation', 'texture', 'composé', 'pollué', 'Début', 'des', 'briquaillon

In [54]:
#TODO: classify words in each category (may be based on a list of common words in these categories)
#idea : load info in a file that save words if there are not in the list (with possibility to add or not)

sep_common=['et', 'par', 'dans', 'scories'] #just an example
sep_lex=[]
#colour_lex=[]
#litho_lex=[]
#state_lex=[]
#qtity_lex=[]
#other_lex=[]

for w in lexic:
    if w in sep_common: 
        sep_lex=sep_lex+[w]
print(sep_lex) #will be used for generate a striplog.Lexicon as a file.py

['scories', 'dans']


--------------------------------------------------------------------------------------------------------

## 2-Database MEMORIS3.xlsx
* **Feuille : 'PROFILS_SOL'**

In [55]:
tmp_dir='../../CF_data/synthese/Result_traitem/database_Memoris3/'
sheet='PROFILS_SOL'

In [56]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. '+
                        'Siterem - 2017/Database MEMORIS3.xlsx', sheet_name='PROFILS_SOL')#, skiprows=2)
gdf_viewer(tmp_for)

Rows : 2041, columns : 16


interactive(children=(IntSlider(value=10, description='rows', max=2041, min=10, readout=False), IntSlider(valu…

In [57]:
tmp_for = tmp_for[['Date', 'N°', 'Id', 'Profondeur', 'Description', 'Piézo', 'Unnamed: 6',
                             'Gouge Ø75', 'MFT Ø145', 'carottier', 'tarrière', 'Liner Ø60']] 

In [58]:
tmp_for.rename({'Date':'Date_ouv', 'N°':'Ref', 'Id':'idx', 'Piézo':'Type', 'Unnamed: 6':'Societe',
                'MFT Ø145':'MFT_145', 'Gouge Ø75':'Gouge_75', 'Liner Ø60': 'Liner_60'}, axis=1, inplace=True)

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [59]:
print(list(set(tmp_for['Date_ouv'].apply(lambda x: x.year if not pd.isnull(x) else x))))

[NaT, 2009, 2010, 2015]


In [60]:
tmp_for.loc[tmp_for.fillna('').query("Societe.str.contains('x|X')").index, 'Type']='X'

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [61]:
tmp_for.loc[tmp_for.fillna('').query("Gouge_75.str.contains('SBS|SITER')").index, 'Societe']='SBS Environnement'
tmp_for.loc[tmp_for.fillna('').query("Gouge_75.str.contains('SBS|SITER')").index, 'Gouge_75']=''

In [62]:
for i in range(len(tmp_for['Date_ouv'])-1):
    if not pd.isnull(tmp_for.loc[i, 'Date_ouv']) and pd.isnull(tmp_for.loc[i+1, 'Date_ouv']):
        tmp_for.loc[i+1, 'Date_ouv']=tmp_for.loc[i, 'Date_ouv']
        
    if not pd.isnull(tmp_for.loc[i, 'Societe']) and pd.isnull(tmp_for.loc[i+1, 'Societe']):
        tmp_for.loc[i+1, 'Societe']=tmp_for.loc[i, 'Societe']
        
    if not pd.isnull(tmp_for.loc[i, 'Type']) and pd.isnull(tmp_for.loc[i+1, 'Type']) and \
       tmp_for.loc[i, 'Ref']==tmp_for.loc[i+1, 'Ref']:
        tmp_for.loc[i+1, 'Type']=tmp_for.loc[i, 'Type']

In [63]:
for i in range(len(tmp_for['idx'])-1):
    if tmp_for.loc[i,'Ref']==tmp_for.loc[i+1,'Ref'] and not pd.isnull(tmp_for.loc[i, 'Profondeur'])\
    and re.findall('Forage',tmp_for.loc[i, 'Profondeur']):
        tmp_for.loc[i+1,'idx']=tmp_for.loc[i, 'Profondeur'][0]+str(tmp_for.loc[i, 'Ref'])
        w=tmp_for.loc[i, 'Profondeur'][0]
    elif tmp_for.loc[i,'Ref']==tmp_for.loc[i+1,'Ref'] and not pd.isnull(tmp_for.loc[i, 'Profondeur']):
        tmp_for.loc[i+1,'idx']=w+str(tmp_for.loc[i, 'Ref'])
    
    if tmp_for.loc[i,'Ref']==tmp_for.loc[i+1,'Ref'] and not pd.isnull(tmp_for.loc[i, 'Profondeur'])\
    and re.findall('Tranch',tmp_for.loc[i, 'Profondeur']):
        tmp_for.loc[i+1,'idx']=tmp_for.loc[i, 'Profondeur'][0]+str(tmp_for.loc[i, 'Ref'])
        w=tmp_for.loc[i, 'Profondeur'][0]
    elif tmp_for.loc[i,'Ref']==tmp_for.loc[i+1,'Ref'] and not pd.isnull(tmp_for.loc[i, 'Profondeur']):
        tmp_for.loc[i+1,'idx']=w+str(tmp_for.loc[i, 'Ref'])
"""      
    if tmp_for.loc[i,'Ref']==tmp_for.loc[i+1,'Ref'] and not pd.isnull(tmp_for.loc[i, 'Profondeur'])\
    and re.findall('Moni',tmp_for.loc[i, 'Profondeur']):
        tmp_for.loc[i+1,'idx']=tmp_for.loc[i, 'Profondeur'][0]+str(tmp_for.loc[i, 'Ref'])
        w=tmp_for.loc[i, 'Profondeur'][0]
    elif tmp_for.loc[i,'Ref']==tmp_for.loc[i+1,'Ref'] and not pd.isnull(tmp_for.loc[i, 'Profondeur']):
        tmp_for.loc[i+1,'idx']=w+str(tmp_for.loc[i, 'Ref'])"""

"      \n    if tmp_for.loc[i,'Ref']==tmp_for.loc[i+1,'Ref'] and not pd.isnull(tmp_for.loc[i, 'Profondeur'])    and re.findall('Moni',tmp_for.loc[i, 'Profondeur']):\n        tmp_for.loc[i+1,'idx']=tmp_for.loc[i, 'Profondeur'][0]+str(tmp_for.loc[i, 'Ref'])\n        w=tmp_for.loc[i, 'Profondeur'][0]\n    elif tmp_for.loc[i,'Ref']==tmp_for.loc[i+1,'Ref'] and not pd.isnull(tmp_for.loc[i, 'Profondeur']):\n        tmp_for.loc[i+1,'idx']=w+str(tmp_for.loc[i, 'Ref'])"

In [64]:
tmp_for['Ref']=tmp_for['idx'].apply(lambda x : x if re.findall('F|T', str(x)) else '')
tmp_for['Ref']=tmp_for['idx'].apply(lambda x : x.replace('Monito ', 'Mon') if re.findall('Monit', str(x)) else x)

<ipython-input-64-9f124d03735a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Ref']=tmp_for['idx'].apply(lambda x : x if re.findall('F|T', str(x)) else '')
<ipython-input-64-9f124d03735a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Ref']=tmp_for['idx'].apply(lambda x : x.replace('Monito ', 'Mon') if re.findall('Monit', str(x)) else x)


In [65]:
tmp_for['Type']=tmp_for['Type'].apply(lambda x: 'Piezo' if not pd.isnull(x) else '')

<ipython-input-65-b3f980806ea8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Type']=tmp_for['Type'].apply(lambda x: 'Piezo' if not pd.isnull(x) else '')


In [66]:
tmp_for['Ref']=tmp_for['Ref'].apply(lambda x: re.sub("\.1","a",str(x)) if re.search(r"\.1", str(x)) else x)
tmp_for['Ref']=tmp_for['Ref'].apply(lambda x: re.sub("\.2","b",str(x)) if re.search(r"\.2", str(x)) else x)
tmp_for['Ref']=tmp_for['Ref'].apply(lambda x: re.sub("\.3","c",str(x)) if re.search(r"\.3", str(x)) else x)
tmp_for['Ref']=tmp_for['Ref'].apply(lambda x: re.sub("\.4","d",str(x)) if re.search(r"\.4", str(x)) else x)

<ipython-input-66-9d4c7d510ecf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Ref']=tmp_for['Ref'].apply(lambda x: re.sub("\.1","a",str(x)) if re.search(r"\.1", str(x)) else x)
<ipython-input-66-9d4c7d510ecf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Ref']=tmp_for['Ref'].apply(lambda x: re.sub("\.2","b",str(x)) if re.search(r"\.2", str(x)) else x)
<ipython-input-66-9d4c7d510ecf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [69]:
gen_id_dated(tmp_for, ref_col='Ref', date_col='Date_ouv')

Generation of ID-dated...
Using column ' Date_ouv ' in the (geo)dataframe !
Process ended, check you (geo)dataframe


/home/su530201/PycharmProjects/GSDMA/utils/io.py:650: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['ID_date'] = gdf[date_col].apply(lambda x : str(x.year) + '-' if not pd.isnull(x) else '') + gdf[ref_col].apply(lambda x : str(x) if not pd.isnull(x) else '')


In [70]:
tmp_for.loc[tmp_for.query('Profondeur!=Profondeur' ).index,'Profondeur']=''

In [71]:
tmp_for['Method']=''
            
for i in range(len(tmp_for['Method'])):
    if not pd.isnull(tmp_for.loc[i, 'Gouge_75']) : tmp_for.loc[i, 'Method']='Gouge_75'
    if not pd.isnull(tmp_for.loc[i, 'MFT_145']) : tmp_for.loc[i, 'Method']='MFT_145'
    if not pd.isnull(tmp_for.loc[i, 'Liner_60']) : tmp_for.loc[i, 'Method']='Liner_60'
    if not pd.isnull(tmp_for.loc[i, 'carottier']) : tmp_for.loc[i, 'Method']='carrotier'
    if not pd.isnull(tmp_for.loc[i, 'tarrière']) : tmp_for.loc[i, 'Method']='tarrière'

<ipython-input-71-6af8b8a7f004>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Method']=''


In [72]:
tmp_for.drop(tmp_for.query('Profondeur.str.contains("Forage") and Profondeur!="Forage bloqué"', engine='python').index, inplace=True)
tmp_for.drop(tmp_for.query('Profondeur.str.contains("Tranc") and Profondeur!="Tranchée bloqué"', engine='python').index, inplace=True)
tmp_for.drop(tmp_for.query('Profondeur.str.contains(".orage|..ranch", regex=True)', engine='python').index, inplace=True)
tmp_for.drop(tmp_for.fillna('').query('Description.str.contains("^.orage bloq|^.ranc.* bloq|^.*efus", regex=True)', engine='python').index, inplace=True)
tmp_for.drop(tmp_for.query('Ref!=Ref').index, inplace=True)
tmp_for.drop(columns=['MFT_145','Gouge_75','Liner_60', 'carottier', 'tarrière', 'idx'], axis=1, inplace=True)
tmp_for.reset_index(drop=True, inplace=True)

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [73]:
tmp_for['Litho_top'] = tmp_for['Profondeur'].apply(lambda x: x.replace(',','.').split('-')[0].strip(' m'))
tmp_for['Litho_base'] = tmp_for['Profondeur'].apply(lambda x: x.replace(',','.').split('-')[-1].strip(' m'))

<ipython-input-73-d159d790a2d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Litho_top'] = tmp_for['Profondeur'].apply(lambda x: x.replace(',','.').split('-')[0].strip(' m'))
<ipython-input-73-d159d790a2d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_for['Litho_base'] = tmp_for['Profondeur'].apply(lambda x: x.replace(',','.').split('-')[-1].strip(' m'))


In [74]:
tmp_for.rename({'Ref':'ID'}, axis=1, inplace=True)
if 'Profondeur' in tmp_for.columns: tmp_for.drop(columns=['Profondeur'], axis=1, inplace=True)

In [75]:
trenches=tmp_for.loc[tmp_for.query('ID_date.str.contains("T")', engine='python').index]
boreholes=tmp_for.loc[tmp_for.query('ID_date.str.contains("F")', engine='python').index]

In [76]:
len(boreholes), len(trenches)

(1588, 40)

In [77]:
gdf_viewer(tmp_for, un_val='ID')

Rows : 1629, columns : 9, Unique on 'ID': 299


interactive(children=(IntSlider(value=10, description='rows', max=1629, min=10, readout=False), IntSlider(valu…

In [78]:
source_litho=tmp_for.copy()

In [79]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
tmp_for.to_csv(tmp_dir+sheet+'.csv', index=False)
boreholes.to_csv(tmp_dir+sheet+'_BH.csv', index=False)
trenches.to_csv(tmp_dir+sheet+'_TRCH.csv', index=False)
source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False) #all lithologies or descriptions data in the source

* **Feuille : 'DONNEES PIEZOS'**

In [80]:
tmp_dir='../../CF_data/synthese/Result_traitem/database_Memoris3/'
sheet='Donnees_piezos'

In [81]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. '+
                        'Siterem - 2017/Database MEMORIS3.xlsx', sheet_name='DONNEES PIEZOS', skiprows=2)
gdf_viewer(tmp_for)

Rows : 147, columns : 186


interactive(children=(IntSlider(value=10, description='rows', max=147, min=10, readout=False), IntSlider(value…

In [82]:
tmp_survey=tmp_for[[tmp_for.columns.to_list()[1]]+tmp_for.columns.to_list()[13:17]]
tmp_for=tmp_for[tmp_for.columns.to_list()[1:13]+tmp_for.columns.to_list()[17:22]]

In [83]:
tmp_for.rename(columns={'Campagne':'Societe','N_piezo.':'ID','X [m]':'X','Y [m]':'Y','Z tête PZ [m]':'Z',
                        'Zsol [m]':'Zsol', 'Prof_PZ [m]':'Long_Pz','Section_crépinée [m]':'Long_Crep',
                        'Diamètre_int [m]':'Diam_Pz_int','Surnageant [cm]':'Surnageant','Sousnageant [cm]':'Sousnageant',
                        'Description éch. \nOd/turb.':'Opacite_eau','Remarques':'Organoleptique'}, inplace=True)

#tmp_for=tmp_for[['ID','X', 'Y', 'Z', 'Zsol', 'Long_Pz', 'Long_Crep', 'Diam_Pz_int', 'Societe']]#[:130]
tmp_for=tmp_for.query("ID ==ID")
tmp_for.replace('-',np.nan, inplace=True)

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [84]:
tmp_for['Type']=tmp_for['Long_Crep'].apply(lambda x: 'Piezo' if not pd.isnull(x) else '')

In [85]:
gdf_viewer(tmp_for, un_val='ID') # all units in [m]

Rows : 130, columns : 18, Unique on 'ID': 130


interactive(children=(IntSlider(value=10, description='rows', max=130, min=10, readout=False), IntSlider(value…

In [86]:
tmp_for=tmp_for[['ID','Societe','X','Y','Z','Zsol','Type','Aquifère','Long_Pz','Long_Crep','Diam_Pz_int', 'Surnageant', 
         'Sousnageant','Caractéristique', 'Organoleptique', 'Opacite_eau','Zone', 'Sous_zone']]

In [87]:
tmp_survey.rename(columns={'N_piezo.':'ID'}, inplace=True)
tmp_survey=tmp_survey.query("ID==ID")

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [88]:
tmp_mes=pd.DataFrame()
col=tmp_survey.columns.to_list()
mes_id=0

for i in range(len(tmp_survey)):
    for j in range(1,len(col)):
        d=col[j].strip('\n|.1').split('/')
        tmp_mes.loc[mes_id,'Date_Mes']=datetime.date(int(d[2]), int(d[1]), int(d[0]))
        tmp_mes.loc[mes_id,'ID']=str(tmp_survey.loc[i,'ID'])
        
        if j<=2: 
            tmp_mes.loc[mes_id,'Niv_Eau_Pz']=tmp_survey.iloc[i,j]
            tmp_mes.loc[mes_id,'Niv_Eau_sol']=tmp_survey.iloc[i,j+2]
            mes_id+=1

tmp_mes=tmp_mes.sort_values('Date_Mes').reset_index(drop=True)
tmp_mes['Mes_ID']=tmp_mes['ID'].apply(lambda x: 'Mes_'+str(x))
tmp_mes.insert(0, 'Mes_ID', tmp_mes.pop('Mes_ID'))

In [89]:
source_for=tmp_for.copy()
source_survey=tmp_mes.copy()

In [90]:
gdf_viewer(source_survey)

Rows : 261, columns : 5


interactive(children=(IntSlider(value=10, description='rows', max=261, min=10, readout=False), IntSlider(value…

In [91]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

In [92]:
tmp_for.to_csv(tmp_dir+sheet+'_BH.csv', index=False)
tmp_mes.to_csv(tmp_dir+sheet+'_SRV.csv', index=False)

In [93]:
source_for.to_csv(tmp_dir+'source_Boreholes.csv', index=False) #all boreholes data in the source
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False) #all lithologies or descriptions data in the source
#source_ech.to_csv(tmp_dir+'Source_Samples.csv', index=False) #all Samples and organoleptic data in the source
#source_analysis.to_csv(tmp_dir+'source_Analysis.csv', index=False) #all analysis data in the source
source_survey.to_csv(tmp_dir+'source_Survey.csv', index=False) #all parameters' survey data in the source

* **Feuille : 'DRAINS ET PIEZOS ENEL'**

In [94]:
tmp_dir='../../CF_data/synthese/Result_traitem/database_Memoris3/'
sheet='Drains_Pz_ENEL'

In [95]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. Siterem - 2017/Database MEMORIS3.xlsx', 
                        sheet_name='DRAINS ET PIEZOS ENEL', skiprows=1)
gdf_viewer(tmp_for)

Rows : 147, columns : 68


interactive(children=(IntSlider(value=10, description='rows', max=147, min=10, readout=False), IntSlider(value…

In [96]:
tmp_for.rename(columns={'N°':'ID', 'Date ':'Date_prv','Hauteur de la chambre ':'Ht_Chbre','T':'Temp'}, inplace=True)

tmp_mes=tmp_for[tmp_for.columns.to_list()[:2]+tmp_for.columns.to_list()[10:-2]]
tmp_for=tmp_for[tmp_for.columns.to_list()[:7]]

In [97]:
tmp_for.columns.to_list()[:12]

['ID', 'Date_prv', 'ETUDE', 'X', 'Y', 'Zsol', 'Ht_Chbre']

In [98]:
tmp_for=tmp_for[['ID', 'X', 'Y', 'Zsol', 'Ht_Chbre']][2:21]
tmp_for.replace('-',np.nan, inplace=True)
tmp_for.drop(index=[5], inplace=True)
tmp_for['Type']='Piezo'
#tmp_for['Zsol']=tmp_for['Zsol'].apply(lambda x: x if not pd.isnull(x) else np.nan)

gdf_viewer(tmp_for, un_val='ID')

Rows : 18, columns : 6, Unique on 'ID': 18


interactive(children=(IntSlider(value=10, description='rows', max=18, min=10, readout=False), IntSlider(value=…

In [99]:
tmp_survey=tmp_mes.query('ID==ID').reset_index(drop=True)

In [100]:
tmp_mes=pd.DataFrame()
col=[tmp_survey.columns.to_list()[2]]+tmp_survey.columns.to_list()[4:6]
mes_id=0
d=['01/10/2013','01/11/2015','01/12/2016'] # 01/11/2015 added by me

for i in range(len(tmp_survey)):
    k=0
    for j in col:
        tmp_mes.loc[mes_id,'Mes_ID']='Mes_'+str(mes_id)
        tmp_mes.loc[mes_id,'Date_Mes']=datetime.date(int(d[k].split('/')[2]), int(d[k].split('/')[1]), 
                                                             int(d[k].split('/')[0]))
        tmp_mes.loc[mes_id,'ID']=str(tmp_survey.loc[i,'ID'])
        tmp_mes.loc[mes_id,'Niv_Eau_sol']=tmp_survey.loc[i,j]
                
        if tmp_mes.loc[mes_id, 'Date_Mes']==tmp_survey.loc[i, 'Date_prv'] and \
        tmp_mes.loc[mes_id, 'ID']==tmp_survey.loc[i, 'ID']:
            tmp_mes.loc[mes_id, tmp_survey.columns.to_list()[6:-1]]=tmp_survey.iloc[i, 6:-1]
        
        mes_id+=1
        k+=1
tmp_mes.replace('-', np.nan, inplace=True)

In [101]:
tmp_mes['line_na']=False
for i in range(len(tmp_mes)):
    verif=True
    for j in tmp_mes.columns.to_list()[3:-1]:
        if not pd.isnull(tmp_mes.loc[i,j]): verif=False
    
    tmp_mes.loc[i, 'line_na']=verif

In [102]:
tmp_mes=tmp_mes.query('line_na==False')
tmp_mes.reset_index(drop=True, inplace=True)
tmp_mes.drop('line_na', axis=1, inplace=True)
tmp_mes['Mes_ID']=tmp_mes['ID'].apply(lambda x: 'Mes_'+x)
tmp_mes['CE']=tmp_mes['CE'].apply(lambda x: x/1000 if not pd.isnull(x) else x)

In [103]:
merge_tmp, error_df=gdf_merger(source_for, tmp_for, 'outer', 'ID')

In [104]:
source_for=merge_tmp.copy()

In [105]:
tmp_an=tmp_mes[tmp_mes.columns.to_list()[1:3]+tmp_mes.columns.to_list()[9:]]
tmp_an['Type_ech']='Eau'
tmp_an.insert(tmp_an.columns.to_list().index('ID')+1, 'Type_ech', tmp_an.pop('Type_ech'))
tmp_mes=tmp_mes[tmp_mes.columns.to_list()[:9]]

<ipython-input-105-f3cbae7f0ac4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_an['Type_ech']='Eau'


In [106]:
merge_tmp, error_df=gdf_merger(source_survey, tmp_mes, 'outer', 'ID', fcol='Mes_ID')

In [107]:
source_survey=merge_tmp.copy()

In [108]:
source_analysis=tmp_an.copy()

In [109]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

In [110]:
tmp_for.to_csv(tmp_dir+sheet+'_BH.csv', index=False)
tmp_mes.to_csv(tmp_dir+sheet+'_SRV.csv', index=False)
tmp_an.to_csv(tmp_dir+sheet+'_ANL.csv', index=False)

In [111]:
source_for.to_csv(tmp_dir+'source_Boreholes.csv', index=False) #all boreholes data in the source
#source_litho.to_csv(tmp_dir+'source_Lithology.csv', index=False) #all lithologies or descriptions data in the source
#source_ech.to_csv(tmp_dir+'Source_Samples.csv', index=False) #all Samples and organoleptic data in the source
source_analysis.to_csv(tmp_dir+'source_Analysis.csv', index=False) #all samples and pollutants data in the source
source_survey.to_csv(tmp_dir+'source_Survey.csv', index=False) #all parameters' survey data in the source

* **Feuille : 'RESULTS_EAU' (F)**

In [112]:
tmp_dir='../../CF_data/synthese/Result_traitem/database_Memoris3/'
sheet='RESULTS_EAU'

In [113]:
tmp_for = pd.read_excel('../../CF_data/Data_UMONS/Contamination/Doc_SITEREM/Rapport de synthèse des études de sol et des eaux souterraines. Siterem - 2017/Database MEMORIS3.xlsx', 
                        sheet_name='RESULTS_EAU', skiprows=1)
gdf_viewer(tmp_for, rows=5)

Rows : 204, columns : 185


interactive(children=(IntSlider(value=5, description='rows', max=204, min=5, readout=False), IntSlider(value=1…

In [114]:
tmp_for.drop(0, inplace=True)
tmp_for.reset_index(drop=True, inplace=True)

In [115]:
tmp_for.rename(columns={'Campagne':'Societe','N_piezo.':'ID','Z tête PZ':'Z','Zsol':'Zsol', 'Prof_PZ':'Long_Pz',
                        'Section_crépinée':'Long_Crep','Diamètre_int':'Diam_Pz_int','Surnageant':'Surnageant',
                        'Sousnageant':'Sousnageant','Description éch.':'Opacite_eau','Remarques':'Organoleptique',
                        'Aquifère_échantillonné':'Aquifère'}, inplace=True)

#tmp_for=tmp_for[['ID','X', 'Y', 'Z', 'Zsol', 'Long_Pz', 'Long_Crep', 'Diam_Pz_int', 'Societe']]#[:130]
tmp_for=tmp_for.query("ID ==ID")
tmp_for.replace('-',np.nan, inplace=True)

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/frame.py:4521: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [116]:
tmp_survey=tmp_for[[tmp_for.columns.to_list()[0]]+tmp_for.columns.to_list()[12:16]+tmp_for.columns.to_list()[21:26]]
tmp_an=tmp_for[[tmp_for.columns.to_list()[0]]+tmp_for.columns.to_list()[26:]]
tmp_for=tmp_for[tmp_for.columns.to_list()[:12]+tmp_for.columns.to_list()[16:21]]

In [117]:
tmp_for['Type']=tmp_for['Long_Crep'].apply(lambda x: 'Piezo' if not pd.isnull(x) else '')
tmp_for.insert(8, 'Type', tmp_for.pop('Type'))
tmp_for['Surnageant']=tmp_for['Surnageant'].apply(lambda x: x/100) # to express value in meter unit
tmp_for['Sousnageant']=tmp_for['Sousnageant'].apply(lambda x: x/100)

In [118]:
gdf_viewer(tmp_for, un_val='ID', rows=5) # all units in [m]

Rows : 130, columns : 18, Unique on 'ID': 130


interactive(children=(IntSlider(value=5, description='rows', max=130, min=5, readout=False), IntSlider(value=1…

In [119]:
tmp_mes=pd.DataFrame()
col=tmp_survey.columns.to_list()[5:]
mes_id=0
d=['27/04/2010', '08/09/2010']

for i in range(len(tmp_survey)):
    for k in [0,1]:
        tmp_mes.loc[mes_id,'Mes_ID']='Mes_'+str(mes_id)
        tmp_mes.loc[mes_id,'ID']=str(tmp_survey.loc[i,'ID'])
        
        dt=d[k].split('/')
        tmp_mes.loc[mes_id,'Date_Mes']=datetime.date(int(dt[2]), int(dt[1]),int(dt[0]))
        tmp_mes.loc[mes_id,'Niv_Eau_Pz']=tmp_survey.iloc[i,k+1]
        tmp_mes.loc[mes_id,'Niv_Eau_sol']=tmp_survey.iloc[i,k+3]
        tmp_mes.loc[mes_id, col]=list(tmp_survey.loc[i,col])
        mes_id+=1          
        
tmp_mes.replace('-', np.nan, inplace=True)
#tmp_mes=tmp_mes.sort_values('Date_Mes').reset_index(drop=True)
tmp_mes['Mes_ID']=tmp_mes['ID'].apply(lambda x: 'Mes_'+str(x))
tmp_mes.insert(0, 'Mes_ID', tmp_mes.pop('Mes_ID'))

In [120]:
q=tmp_mes.query('Niv_Eau_Pz.isnull() and Niv_Eau_sol.isnull()').index
tmp_mes.drop(q, inplace=True)
tmp_mes['CE']=tmp_mes['CE'].apply(lambda x: x/1000 if not pd.isnull(x) else x)

In [121]:
tmp_an['line_na']=False
for i in range(len(tmp_an)):
    verif=True
    for j in tmp_an.columns.to_list()[3:-1]:
        if not pd.isnull(tmp_an.loc[i,j]): verif=False
    
    tmp_an.loc[i,'line_na']=verif

In [122]:
tmp_an=tmp_an.query('line_na==False')
tmp_an.reset_index(drop=True, inplace=True)
tmp_an.drop('line_na', axis=1, inplace=True)
tmp_an.insert(1, 'Type_ech', 'Eau')
tmp_an['Anl_ID']=tmp_an['ID'].apply(lambda x: 'Anl_'+str(x))
tmp_an.insert(0, 'Anl_ID', tmp_an.pop('Anl_ID'))

**Revoir bien cette partie à tête reposée (fusion sur la source)**

In [123]:
merge_tmp, error_df=gdf_merger(source_survey, tmp_mes, 'outer', 'ID', fcol='Mes_ID')

Ambiguous values in both columns compared, change it manually !
Columns ['Niv_Eau_Pz_x', 'Niv_Eau_Pz_y', 'Niv_Eau_sol_x', 'Niv_Eau_sol_y', 'Date_Mes_x', 'Date_Mes_y'] [1, 2, 5, 6, 9, 10, 13, 14, 17, 18, 21, 22, 25, 26, 29, 30, 33, 34, 37, 38, 41, 42, 45, 46, 51, 52, 58, 60, 63, 64, 67, 68, 71, 72, 75, 76, 79, 80, 83, 84, 87, 88, 91, 92, 97, 98, 101, 103, 104, 107, 108, 111, 112, 115, 116, 119, 120, 131, 132, 135, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 181, 184, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 230, 231, 234, 235, 238, 239, 242, 243, 246, 247, 250, 251, 254, 255, 260, 261, 264, 265, 268, 269, 272, 273, 276, 277, 280, 281, 284, 285, 288, 290, 291, 294, 295, 298, 299, 302, 303, 306, 307, 310, 311, 314, 315, 318, 319, 322, 323, 326, 327, 332, 333, 336, 337, 340, 341, 344, 345, 350, 351, 354, 355, 358, 359, 362, 363, 366, 367, 372, 373, 378, 379

In [124]:
gdf_viewer(merge_tmp)

Rows : 431, columns : 17


interactive(children=(IntSlider(value=10, description='rows', max=431, min=10, readout=False), IntSlider(value…

In [125]:
gdf_viewer(tmp_an)

Rows : 119, columns : 162


interactive(children=(IntSlider(value=10, description='rows', max=119, min=10, readout=False), IntSlider(value…

In [126]:
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

In [127]:
tmp_for.to_csv(tmp_dir+sheet+'_BH.csv', index=False)
tmp_mes.to_csv(tmp_dir+sheet+'_SRV.csv', index=False)
tmp_an.to_csv(tmp_dir+sheet+'_ANL.csv', index=False)

* **Feuille : 'RESULTS_SOL'**

============================================ ICI ===========================================================

============================================ draft ===========================================================

* **Feuille : 'PROFILS_SOL'**
DONNEES PIEZOS, DRAINS ET PIEZOS ENEL, RESULTS_SOL, RESULTS_EAU, Normes

============================================ old codes ===========================================

pour la suite extraire les fichiers avec XY

/////////////////////////////----- Brouillon de codes -----//////////////////////////////////////////////////////////////

////////////////////////////////////////-------------------------------------------------------------------------------///////////////////////////////////////////////

## Extraction des dates de forages du type de forage et du foreur de l'étude de caractérisation

### Informations sur les forages de l'étude de caractérisation de 2010